In [4]:
#Youtube影片標題爬蟲

import requests
from bs4 import BeautifulSoup
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.116 Safari/537.36'}
url = 'https://www.youtube.com/'

resp = requests.get(url,headers=headers)
soup = BeautifulSoup(resp.text, 'html.parser')
target=soup.find_all('a')
#存成txt檔案
txt = open('video-title.txt', 'w', encoding = 'UTF-8')
for i in target:
    f=i.get_text().strip() #取得文字、去除左右的空格
    txt.write(f)           #寫入文字
    txt.write('\n')        #換行
txt.close()                #關閉檔案

In [1]:
#下載 Yahoo 頭條新聞
import requests
from bs4 import BeautifulSoup

# 下載 Yahoo 首頁內容
r = requests.get('https://tw.yahoo.com/')

# 確認是否下載成功
if r.status_code == requests.codes.ok:
  # 以 BeautifulSoup 解析 HTML 程式碼
  soup = BeautifulSoup(r.text, 'html.parser')

  # 以 CSS 的 class 抓出各類頭條新聞
  stories = soup.find_all('a', class_='story-title')
  for s in stories:
    # 新聞標題
    print("標題：" + s.text)
    # 新聞網址
    print("網址：" + s.get('href'))

標題：境外病例不斷「陸恐打延長賽」
網址：https://tw.news.yahoo.com/%E5%96%AE%E6%97%A5%E7%A2%BA%E8%A8%BA%E7%A0%B4%E7%99%BE-%E9%99%B8%E6%81%90%E6%89%93%E5%BB%B6%E9%95%B7%E8%B3%BD-165116884.html
標題：聚餐返家一夜未醒 上校猝死
網址：https://tw.news.yahoo.com/%E5%BE%8C%E6%8C%87%E9%83%A8%E5%89%AF%E6%97%85%E9%95%B7%E8%81%9A%E9%A4%90%E8%BF%94%E5%AE%B6%E5%BE%8C%E7%8C%9D%E6%AD%BB%E9%81%B2%E6%9C%AA%E8%B5%B7%E5%BA%8A%E5%AE%B6%E5%B1%AC%E7%99%BC%E7%8F%BE%E8%BA%AB%E9%AB%94%E5%B7%B2%E5%86%B0%E5%86%B7-084556825.html
標題：華航改名？招牌就值10億美元
網址：https://tw.news.yahoo.com/%E8%8F%AF%E8%88%AA%E6%8B%9B%E7%89%8C%E5%80%BC10%E5%84%84%E7%BE%8E%E5%85%83-%E6%94%B9%E5%90%8D%E8%88%AA%E6%AC%8A%E8%AE%8A%E6%95%B8%E5%A4%A7-225614359.html
標題：台商拒繳百萬罰金 9土地全查封
網址：https://tw.news.yahoo.com/%E5%8F%B0%E5%95%86%E9%81%95%E5%8F%8D%E6%AA%A2%E7%96%AB%E6%8B%92%E7%B9%B3100%E8%90%AC-%E9%81%AD%E6%9F%A5%E5%B0%819%E7%AD%86%E5%9C%9F%E5%9C%B0%E9%99%90%E5%88%B6%E5%87%BA%E5%A2%83-075701529.html
標題：2道禁令 看教部怎輕賤大學自治
網址：https://tw.news.yahoo.com/%E5%BE%9E%E5%85%A9%E9%81%93%E7%A6%81%E4%BB%

In [3]:
import requests
import time
from bs4 import BeautifulSoup
import os
import re
import urllib.request
import json


PTT_URL = 'https://www.ptt.cc'

#取得網頁文件的function
def get_web_page(url):
    time.sleep(0.5)  # 每次爬取前暫停 0.5 秒以免被 PTT 網站判定為大量惡意爬取
    resp = requests.get(
        url=url,
        cookies={'over18': '1'}
    )
    if resp.status_code != 200:
        print('Invalid url:', resp.url)
        return None
    else:
        return resp.text


def get_articles(dom, date):
    soup = BeautifulSoup(dom, 'html.parser')

    # 取得上一頁的連結
    paging_div = soup.find('div', 'btn-group btn-group-paging')
    prev_url = paging_div.find_all('a')[1]['href']

    articles = []  # 儲存取得的文章資料
    divs = soup.find_all('div', 'r-ent')
    for d in divs:
        if d.find('div', 'date').string.strip() == date:  # 發文日期正確
            # 取得推文數
            push_count = 0
            if d.find('div', 'nrec').string:
                try:
                    push_count = int(d.find('div', 'nrec').string)  # 轉換字串為數字
                except ValueError:  # 若轉換失敗，不做任何事，push_count 保持為 0
                    pass

            # 取得文章連結及標題
            if d.find('a'):  # 有超連結，表示文章存在，未被刪除
                href = d.find('a')['href']
                title = d.find('a').string
                articles.append({
                    'title': title,
                    'href': href,
                    'push_count': push_count
                })
    return articles, prev_url#回傳這一頁的文章和上一頁


def parse(dom):
    soup = BeautifulSoup(dom, 'html.parser')
    links = soup.find(id='main-content').find_all('a')
    img_urls = []
    for link in links:
        if re.match(r'^https?://(i.)?(m.)?imgur.com', link['href']):
            img_urls.append(link['href'])
    return img_urls


def save(img_urls, title):
    if img_urls:
        try:
            dname = title.strip()  # 用 strip() 去除字串前後的空白
            os.makedirs(dname)
            for img_url in img_urls:
                if img_url.split('//')[1].startswith('m.'):
                    img_url = img_url.replace('//m.', '//i.')
                if not img_url.split('//')[1].startswith('i.'):
                    img_url = img_url.split('//')[0] + '//i.' + img_url.split('//')[1]
                if not img_url.endswith('.jpg'):
                    img_url += '.jpg'
                fname = img_url.split('/')[-1]
                urllib.request.urlretrieve(img_url, os.path.join(dname, fname))
        except Exception as e:
            print(e)
if __name__ == '__main__':
    current_page = get_web_page(PTT_URL + '/bbs/Beauty/index.html')
    if current_page:
        articles = []  # 全部的今日文章
        date = time.strftime("%m/%d").lstrip('0')  # 今天日期, 去掉開頭的 '0' 以符合 PTT 網站格式
        current_articles, prev_url = get_articles(current_page, date)  # 目前頁面的今日文章
        while current_articles:  # 若目前頁面有今日文章則加入 articles，並回到上一頁繼續尋找是否有今日文章
            articles += current_articles
            current_page = get_web_page(PTT_URL + prev_url)
            current_articles, prev_url = get_articles(current_page, date)

        # 已取得文章列表，開始進入各文章讀圖
        for article in articles:
            print('Processing', article)
            page = get_web_page(PTT_URL + article['href'])
            if page:
                img_urls = parse(page)
                save(img_urls, article['title'])
                article['num_image'] = len(img_urls)

        # 儲存文章資訊
        with open('data.json', 'w', encoding='utf-8') as f:
            json.dump(articles, f, indent=2, sort_keys=True, ensure_ascii=False)

# '''
# json.loads()是将str转化成dict格式，json.dumps()是将dict转化成str格式。
# json.load()和json.dump()也是类似的功能，只是与文件操作结合起来了。
# join()： 连接字符串数组。将字符串、元组、列表中的元素以指定的字符(分隔符)连接生成一个新的字符串
# Skipkeys：默认值是False，如果dict的keys内的数据不是python的基本类型(str,unicode,int,long,float,bool,None)，设置为False时，就会报TypeError的错误。此时设置成True，则会跳过这类key
# ensure_ascii：默认值True，如果dict内含有non-ASCII的字符，则会类似\uXXXX的显示数据，设置成False后，就能正常显示
# indent：应该是一个非负的整型，如果是0，或者为空，则一行显示数据，否则会换行且按照indent的数量显示前面的空白，这样打印出来的json数据也叫pretty-printed json
# separators：分隔符，实际上是(item_separator, dict_separator)的一个元组，默认的就是(‘,’,’:’)；这表示dictionary内keys之间用“,”隔开，而KEY和value之间用“：”隔开。
# encoding：默认是UTF-8，设置json数据的编码方式。
# sort_keys：将数据根据keys的值进行排序。(a-z)
# Python strip() 方法用于移除字符串头尾指定的字符（默认为空格或换行符）或字符序列。 注意：该方法只能删除开头或是结尾的字符，不能删除中间部分的字符。
# '''


Processing {'title': '[正妹] 健身正妹 兇', 'href': '/bbs/Beauty/M.1586837739.A.E4B.html', 'push_count': 0}
Processing {'title': '[神人] 那些年的無名網紅現況 ?', 'href': '/bbs/Beauty/M.1586838094.A.8F5.html', 'push_count': 14}
[WinError 123] 檔案名稱、目錄名稱或磁碟區標籤語法錯誤。: '[神人] 那些年的無名網紅現況 ?'
Processing {'title': '[正妹] 兇屁', 'href': '/bbs/Beauty/M.1586844061.A.508.html', 'push_count': 16}
Processing {'title': '[廣告] 絕世美少女 明里紬(明里つむぎ) ', 'href': '/bbs/Beauty/M.1586845504.A.7BD.html', 'push_count': 27}
Processing {'title': '[正妹] 衣服很好看', 'href': '/bbs/Beauty/M.1586848942.A.70C.html', 'push_count': 4}
Processing {'title': '[正妹] 包口罩', 'href': '/bbs/Beauty/M.1586852305.A.69A.html', 'push_count': 4}
Processing {'title': '[正妹] 正', 'href': '/bbs/Beauty/M.1586852589.A.6CA.html', 'push_count': 2}
Processing {'title': '[正妹]  你是不是想……', 'href': '/bbs/Beauty/M.1586853720.A.516.html', 'push_count': 23}
Processing {'title': '[正妹] 道歉時露出胸部常識吧', 'href': '/bbs/Beauty/M.1586858766.A.E7B.html', 'push_count': 25}
Processing {'title': '[神人] 業

In [4]:
# Selenium是什麼？一句話，自動化測試工具。它支持各種瀏覽器，包括Chrome，Safari，Firefox等主流界面式瀏覽器，如果你在這些瀏覽器裡面安裝
# 一個Selenium的插件，那麼便可以方便地實現Web界面的測試。換句話說叫Selenium支持這些瀏覽器驅動。

# Selenium有兩個版本，目前最新版本是2.53.1（2016/3/22）

# Selenium 2，又名WebDriver，它的主要新功能是集成了Selenium 1.0以及WebDriver（WebDriver曾經是Selenium的競爭對手）也就是說
# Selenium 2是Selenium和WebDriver兩個項目的合併，即Selenium 2兼容硒，它支持Selenium API也支持WebDriver API。

In [6]:
from selenium import webdriver

browser = webdriver.Chrome('/Users/ASUS/chromedriver')
browser.get('http://www.baidu.com/')